### Problem Statement 

###### To build a machine learning/deep learning approach to forecast the total energy demand on an hourly basis for the next 3 years based on past trends

###### Data :
Data have captured the estimated total energy demand from the past 12 years on an hourly basis. Now, the government of Green Energy is looking for a data scientist to understand the data and forecast the total energy demand for the next 3 years based on past trends.

In [44]:
#import libraries
import numpy as np
import pandas as pd
import time

import warnings
warnings.filterwarnings("ignore")

#validation metrics Library
from sklearn. metrics import mean_squared_error
from math import sqrt

In [45]:
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', 500)
# pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from xgboost import XGBRegressor
from xgboost import plot_importance

def plot_features(booster, figsize):    
    fig, ax = plt.subplots(1,1,figsize=figsize)
    return plot_importance(booster=booster, ax=ax)

import time
import sys
import gc

In [46]:
#Train and Test Data

train=pd.read_csv('train_IxoE5JN.csv',parse_dates=['datetime'])
test=pd.read_csv('test_WudNWDM.csv',parse_dates=['datetime'])

In [47]:
train.head(5)

,row_id,datetime,energy
0,1,2008-03-01 00:00:00,1259.985563
1,2,2008-03-01 01:00:00,1095.541500
2,3,2008-03-01 02:00:00,1056.247500
3,4,2008-03-01 03:00:00,1034.742000
4,5,2008-03-01 04:00:00,1026.334500


In [48]:
train.shape

(94992, 3)

In [49]:
test.head(5)

,row_id,datetime
0,94993,2019-01-01 00:00:00
1,94994,2019-01-01 01:00:00
2,94995,2019-01-01 02:00:00
3,94996,2019-01-01 03:00:00
4,94997,2019-01-01 04:00:00


In [50]:
test.shape

(26304, 2)

In [51]:
#Preprocessing the column datetime - splitting into day,month,year,week,quarter,hour

def datetime_feature(df):

    df['Year'] = pd.to_datetime(df['datetime']).dt.year

    df['Month'] = pd.to_datetime(df['datetime']).dt.month

    df['Day'] = pd.to_datetime(df['datetime']).dt.day

    df['Dayofweek'] = pd.to_datetime(df['datetime']).dt.dayofweek

    df['DayOfyear'] = pd.to_datetime(df['datetime']).dt.dayofyear

    df['Week'] = pd.to_datetime(df['datetime']).dt.week

    df['Quarter'] = pd.to_datetime(df['datetime']).dt.quarter 

    df['Semester'] = np.where(df['Quarter'].isin([1,2]),1,2)

    df['Is_weekend'] = np.where(df['Dayofweek'].isin([5,6]),1,0)

    df['Is_weekday'] = np.where(df['Dayofweek'].isin([0,1,2,3,4]),1,0)

#     df['Days_in_month'] = pd.to_datetime(df['datetime']).dt.days_in_month
    
    df['Hour'] = pd.to_datetime(df['datetime']).dt.hour

    return df

In [52]:
train_df = datetime_feature(train)

In [53]:
train_df.shape

(94992, 14)

In [54]:
#Check if null value present in traindata

train_df.isna().sum()

row_id           0
datetime         0
energy        1900
Year             0
Month            0
Day              0
Dayofweek        0
DayOfyear        0
Week             0
Quarter          0
Semester         0
Is_weekend       0
Is_weekday       0
Hour             0
dtype: int64

###### Imputing Null values

As our problem statement requires hourly basis forcasting, replacing the null values by mean of target label after grouping them based on hour. 

In [55]:
train_df["energy"] = train_df.groupby("Hour").energy.transform(lambda x : x.fillna(x.mean()))
train_df['energy'].isna().sum()

0

In [56]:
#preprocessing test data

test_df = datetime_feature(test)
test_df.isna().sum()

row_id        0
datetime      0
Year          0
Month         0
Day           0
Dayofweek     0
DayOfyear     0
Week          0
Quarter       0
Semester      0
Is_weekend    0
Is_weekday    0
Hour          0
dtype: int64

In [57]:
#Concatinating train and test data

train_df['train_or_test']='train'
test_df['train_or_test']='test'
df=pd.concat([train,test])

In [58]:
def agg_categorical(df, parent_var, df_name):
    """
    Aggregates the categorical features in a child dataframe
    for each observation of the parent variable.
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    parent_var : string
        The variable by which to group and aggregate the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with aggregated statistics for each observation of the parent_var
        The columns are also renamed and columns with duplicate values are removed.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[parent_var] = df[parent_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    # Remove duplicate columns by values
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

In [59]:
def create_sales_agg_monthwise_features(df, gpby_cols, target_col, agg_funcs):
    '''
    Creates various sales agg features with given agg functions  
    '''
    gpby = df.groupby(gpby_cols)
    newdf = df[gpby_cols].drop_duplicates().reset_index(drop=True)
    for agg_name, agg_func in agg_funcs.items():
        aggdf = gpby[target_col].agg(agg_func).reset_index()
        aggdf.rename(columns={target_col:target_col+'_'+agg_name}, inplace=True)
        newdf = newdf.merge(aggdf, on=gpby_cols, how='left')
    return newdf


In [60]:
 agg_df = create_sales_agg_monthwise_features(df.loc[df.train_or_test=='train', :], 
                                              gpby_cols=['Hour'], 
                                              target_col='energy', 
                                              agg_funcs={'mean':np.mean, 
                                              'median':np.median, 'max':np.max, 
                                              'min':np.min, 'std':np.std})

In [61]:
df=df.merge(agg_df,on=['Hour'],how='left')

In [62]:
df

,row_id,datetime,energy,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Semester,Is_weekend,Is_weekday,Hour,train_or_test,energy_mean,energy_median,energy_max,energy_min,energy_std
0,1,2008-03-01 00:00:00,1259.985563,2008,3,1,5,61,9,1,1,1,0,0,train,1612.395689,1586.16045,3080.4576,1029.7350,303.573289
1,2,2008-03-01 01:00:00,1095.541500,2008,3,1,5,61,9,1,1,1,0,1,train,1528.797757,1502.48560,3041.3108,959.3514,297.176440
2,3,2008-03-01 02:00:00,1056.247500,2008,3,1,5,61,9,1,1,1,0,2,train,1479.056358,1452.36840,3033.6724,904.8438,296.726271
3,4,2008-03-01 03:00:00,1034.742000,2008,3,1,5,61,9,1,1,1,0,3,train,1449.068755,1421.01680,3031.7628,882.7056,297.831180
4,5,2008-03-01 04:00:00,1026.334500,2008,3,1,5,61,9,1,1,1,0,4,train,1441.020137,1409.78325,3067.0904,866.3886,304.067758
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121291,121292,2021-12-31 19:00:00,NaN,2021,12,31,4,365,52,4,2,0,1,19,test,1850.614078,1821.92710,3183.3032,1092.6216,366.587849
121292,121293,2021-12-31 20:00:00,NaN,2021,12,31,4,365,52,4,2,0,1,20,test,1845.669776,1820.71390,3231.5888,1083.7134,352.082043
121293,121294,2021-12-31 21:00:00,NaN,2021,12,31,4,365,52,4,2,0,1,21,test,1841.220023,1819.81565,3231.7252,1098.4428,338.464782
121294,121295,2021-12-31 22:00:00,NaN,2021,12,31,4,365,52,4,2,0,1,22,test,1811.392847,1790.66765,3207.8552,1132.3116,328.747549


In [63]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test','datetime'},axis=1,inplace=True)
test.drop(columns={'train_or_test','datetime'},axis=1,inplace=True)

In [64]:
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [65]:
train.columns

Index(['row_id', 'energy', 'Year', 'Month', 'Day', 'Dayofweek', 'DayOfyear',
       'Week', 'Quarter', 'Semester', 'Is_weekend', 'Is_weekday', 'Hour',
       'energy_mean', 'energy_median', 'energy_max', 'energy_min',
       'energy_std'],
      dtype='object')

In [66]:
train1 = train.loc[:,['Year', 'Month', 'Day', 'Dayofweek', 'DayOfyear',
       'Week', 'Quarter', 'Semester', 'Is_weekend', 'Is_weekday', 'Hour',
       'energy_mean', 'energy_median', 'energy_max', 'energy_min',
       'energy_std']]
train1.shape

(94992, 16)

In [67]:
train_y = train_df.loc[:,['energy']]
train_y.shape

(94992, 1)

In [68]:
#Train and Test split to validate the model training
from sklearn.model_selection import train_test_split

In [69]:
x_train, x_test, y_train, y_test = train_test_split(train1, train_y, test_size=0.3, random_state=42)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(66494, 16) (28498, 16) (66494, 1) (28498, 1)


In [70]:
#Prediction on Test Data
test.columns


Index(['row_id', 'energy', 'Year', 'Month', 'Day', 'Dayofweek', 'DayOfyear',
       'Week', 'Quarter', 'Semester', 'Is_weekend', 'Is_weekday', 'Hour',
       'energy_mean', 'energy_median', 'energy_max', 'energy_min',
       'energy_std'],
      dtype='object')

In [71]:
test_pred = test.loc[:,['row_id']]

In [72]:
test_data = test.loc[:,['Year', 'Month', 'Day', 'Dayofweek', 'DayOfyear',
       'Week', 'Quarter', 'Semester', 'Is_weekend', 'Is_weekday', 'Hour',
       'energy_mean', 'energy_median', 'energy_max', 'energy_min',
       'energy_std']]

##### XGBoost

In [73]:
import xgboost as xgb

In [74]:
# model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=3, 
#                              min_child_weight=1.7817, n_estimators=2200,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5213, silent=1,
#                              random_state =7, nthread = -1,
#                               min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)
# model_xgb.fit(train1, train_y)
# xgbtrain=model_xgb.predict(train1)
# print("xgb_RMSE : %.4g" % np.sqrt(mean_squared_error(train_y, xgbtrain)))


In [75]:
# model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
#                              learning_rate=0.05, max_depth=3, 
#                              min_child_weight=1.7817, n_estimators=2000,
#                              reg_alpha=0.4640, reg_lambda=0.8571,
#                              subsample=0.5, 
#                              random_state =7, nthread = -1)
# model_xgb.fit(train1, train_y)
# xgbtrain=model_xgb.predict(train1)
# print("xgb_RMSE : %.4g" % np.sqrt(mean_squared_error(train_y, xgbtrain))) #gave some good results

In [76]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.8, n_estimators=1500,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5, 
                             random_state =7, nthread = -1)
model_xgb.fit(train1, train_y)
xgbtrain=model_xgb.predict(train1)
print("xgb_RMSE : %.4g" % np.sqrt(mean_squared_error(train_y, xgbtrain)))

xgb_RMSE : 124.6


In [77]:
print(model_xgb.score(train1, train_y))

0.8842092677205474


In [78]:
test_pred['energy'] = model_xgb.predict(test_data)
test_pred.to_csv('mode_xgb_pred.csv', index=False)